### Generating STEM images with drift, as a function of scan angle

To run this notebook, you need:
- Numpy
- Matplotlib 
- scipy
- tqdm
- genSTEM (available from github)
- cupy (optional, speeds things up, but needs CUDA 11.1 (not newer - yet)
- ipympl (to show inline interactive figures in the notebook)

The easiest way to have everything is to create an anaconda environment with the required packages, and
install genSTEM directly from the github. For the best, "widescreen" viewing experience, I recommend jupyter lab.

```bash
conda create --name stem numpy matplotlib scipy tqdm notebook ipympl # jupyterlab
conda activate stem
pip install https://github.com/thomasaarholt/generate_scan_distorted_stem/archive/main.zip
```

In [ ]:
%matplotlib widget
# This thing just saves some whitespace from a bug in tqdm / notebook
from IPython.display import HTML
display(HTML("""
<style>
.jp-OutputArea-prompt:empty {
  padding: 0;
  border: 0;
}
</style>
"""))

In [2]:
from importlib import reload
import numpy as np
import matplotlib.pyplot as plt
from genSTEM import Model, Correct

### Generating images

In [3]:
atoms = Model.get_atoms()

In [7]:
images, scanangles, random_angle = Model.get_rotation_series(atoms, drift_speed=12, vacuum=10, nImages=8, pixel_size=0.2)

  0%|          | 0/8 [00:00<?, ?it/s]

Size: 0.004910656 GB
Shape: (8, 277, 277)


In [8]:
ncols = 4 if len(images) > 4 else len(images)
mult = 12 / ncols
nrows = len(images) // ncols if len(images) % ncols == 0 else len(images) // ncols +1
rowcols = np.array([nrows, ncols])

#### Let's see what the images look like, and what would happen if we tried to rigidly align them!

In [17]:
fig, AX = plt.subplots(*rowcols, figsize=mult*rowcols[::-1])
for ax, img, angle in zip(AX.flatten(), images, scanangles):
    img = Correct.asnumpy(img)
    ax.imshow(img, cmap='jet')
    ax.axis('off')
    ax.set_title(f'{angle}°')

limits = images.min(), images.max()
plt.figure()
img = Correct.create_final_stack_and_average(images, scanangles, 0, 0, print_shifts=True)
img[img == 0] = np.nan
plt.imshow(img, clim=limits, cmap='jet')
plt.title('Registration without Affine Correction')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Registration without Affine Correction')

### Not great. Let's work out what the drift is by transforming by possible drift vectors, and seeing if we get a better correleation!
### Watch the following cell during the running of the cell **after it** - it will live update, but don't trigger the cell after it too quickly, or it'll skip the figure

In [18]:
fig = plt.figure(figsize = (12,4))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
print(f"\nActual angle: {np.round(np.rad2deg(random_angle), 2)}°")
best_angle, best_speed, history = Correct.estimate_drift(images, scanangles, tolerancy_percent=2, normalize_correlation=True, debug=False)
print(f"Missed by {np.round(abs(best_angle - np.rad2deg(random_angle)) % 360,2)}°")


Actual angle: 350.65°
Getting fit with no drift first


Iterating through drift angles:   0%|          | 0/1 [00:00<?, ?it/s]

Iterating through drift speeds:   0%|          | 0/1 [00:00<?, ?it/s]


Iteration #0: Best guess: Angle: 0°, Speed: 0.00e+00. Fit value: 9.61e+06
Limits: [0.0°, 315.0°], [0.00e+00, 3.61e-03]


Iterating through drift angles:   0%|          | 0/8 [00:00<?, ?it/s]

...

Iterating through drift speeds:   0%|          | 0/8 [00:00<?, ?it/s]

Iterating through drift speeds:   0%|          | 0/8 [00:00<?, ?it/s]

Iterating through drift speeds:   0%|          | 0/8 [00:00<?, ?it/s]

Iterating through drift speeds:   0%|          | 0/8 [00:00<?, ?it/s]

Adjusting drift speed limits
Final iteration: Final guess: Angle: -9.46°, Speed: 7.86e-04

Took 24.3 seconds
Missed by 0.11°


In [31]:
imgs = Correct.warp_and_shift_images(images, scanangles, best_speed, best_angle)
fig, AX = plt.subplots(*rowcols, figsize=mult*rowcols[::-1])
for ax, img in zip(AX.flatten(), imgs):
    ax.imshow(Correct.asnumpy(img))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(9,3))

img1 = Correct.create_final_stack_and_average(images, scanangles, 0, 0)
ax1.set_title('Only Registration')
ax1.imshow(img1, cmap='jet')

img2 = Correct.create_final_stack_and_average(images, scanangles, best_speed, best_angle)
ax2.set_title('Affine Transformation\nand Registration')
ax2.imshow(img2, cmap='jet')

img3 = Correct.create_final_stack_and_average(images, scanangles, best_speed, best_angle, subpixel=True)
ax3.set_title('Affine Transformation\nand Subpixel Registration')
ax3.imshow(img3, cmap='jet')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …